In [ ]:
import numpy as np
from sklearn.manifold import MDS
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()

# Make a graph
from linknode_graph_patterns import LinkNodeGraphPatterns
patterns = LinkNodeGraphPatterns()
g         = patterns.createPattern('stanford_facebook_networks')
df        = patterns.nxGraphToPolarsDataFrame(g)

# Determine the distances
dists     = dict(nx.all_pairs_dijkstra_path_length(g))
nodes     = list(dists.keys())
node_to_i = {}
for i in range(len(nodes)): node_to_i[nodes[i]] = i
dissim = np.zeros((len(nodes),len(nodes)))
for _node0_ in g.nodes:
    for _node1_ in g.nodes:
        if _node0_ == _node1_:            continue
        if _node1_ not in dists[_node0_]: continue
        dissim[node_to_i[_node0_]][node_to_i[_node1_]] = dists[_node0_][_node1_]

# Execute MDS
mds           = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
X_transformed = mds.fit_transform(dissim)
pos_mds           = {}
for i in range(len(nodes)): pos_mds[nodes[i]] = [X_transformed[i][0], X_transformed[i][1]]

# Comparison Springs
pos_nx_spring     = nx.spring_layout(g)

# Color the nodes
_node_colors_ = {}
community_i   = 0
for _community_ in nx.community.louvain_communities(g):
    community_i += 1
    for _node_ in _community_: _node_colors_[_node_] = rt.co_mgr.getColor(community_i)

_params_ = {'df':df, 'relationships':[('fm','to')], 'node_color':_node_colors_}
rt.tile([rt.link(pos=pos_mds,       **_params_),
         rt.link(pos=pos_nx_spring, **_params_)])

In [ ]:
import random
#
# Sparse multidimensional scaling using landmark points
# Vin de Silva∗ & Joshua B. Tenenbaum†
# June 30, 2004
#
# Figure 2 // attempted to keep the same nomenclature
#
def landmarkMaxMin(g, 
                   n = 10,  # desired number of landmark points
                   s = 2):  # number of seed points
    nodes = list(g.nodes())
    N     = len(nodes)

    # Choose seeds at random
    l     = [] # landmarks to return
    while len(l) < s:
        seed  = random.choice(nodes)
        if seed not in l: l.append(seed)
    d = {} # d[landmark_point][all_other_points]

    # Determine the shortest paths for the selected nodes
    for i in range(len(l)): d[l[i]] = dict(nx.single_source_shortest_path_length(g, l[i]))

    # Determine the min distance from any of the selected nodes
    m = [(N+1) for i in range(N)]
    for i in range(len(l)):
        for j in range(N):
            if nodes[j] not in d[l[i]]: continue
            m[j] = min(m[j], d[l[i]][nodes[j]])

    # Add the landmark points via MaxMin methodology
    while len(l) < s + n:
        # Determine the max of the m array
        _max_i_ = 0
        for j in range(N):
            if m[_max_i_] < m[j]: _max_i_ = j

        # That's the new landmark -- add it to the list and fill in the distance array
        l.append(nodes[_max_i_])
        d[nodes[_max_i_]] = dict(nx.single_source_shortest_path_length(g, nodes[_max_i_]))

        # Update the mins
        for j in range(N):
            if nodes[j] not in d[nodes[_max_i_]]: continue
            m[j] = min(m[j], d[nodes[_max_i_]][nodes[j]])

    return l

landmarkMaxMin(g)

In [ ]:
l = [1,2,3]
l[1] = 10
l